In [1]:
import file_collecter.tmdb_data_collecter as tdc
import time
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

In [2]:
start_date = '2025-11-01'
end_date = '2025-11-03'

In [1]:
if __name__ == "__main__":
    total_start = time.time()

    # 기간을 3개월 단위로 분할
    periods = tdc.generate_date_periods(start_date, end_date, months=3)

    print(f"\n전체 기간: {start_date} ~ {end_date}")
    print(f"분할 기간: {len(periods)}개")
    print("="*90)

    # ID 데이터를 집합으로 변환하여 중복 제거
    all_ids = set()

    for idx, (period_start, period_end) in enumerate(periods, 1):
        print(f"\n[{idx}/{len(periods)}] 기간: {period_start} ~ {period_end}")

        # 기간내 영화 ID 수집
        current_ids_set = tdc.collect_movie_ids(period_start, period_end)
        # ID를 집합에 추가
        all_ids.update(current_ids_set)

        print(f"해당 기간 내 수집된 ID: {len(current_ids_set):,}개 (누적 ID: {len(all_ids):,}개)")

    # 집합을 리스트로 변환
    id_list = list(all_ids)

    print(f"\n{'='*90}")
    print(f"최종 ID 개수: {len(id_list):,}개")

    print(f"\n{'='*90}")
    print("상세 정보 멀티스레드 수집 시작...\n")

    results = []
    # 3. 상세 정보 수집 단계: ID 리스트를 사용
    with ThreadPoolExecutor(max_workers=30) as executor:
        # 딕셔너리 대신 ID 자체를 future에 매핑
        futures = {executor.submit(tdc.fetch_movie_details, movie_ids): movie_ids for movie_ids in id_list}

        for future in tqdm(as_completed(futures), total=len(futures), desc="영화 상세 수집"):
            detail = future.result()
            if detail:
                results.append(detail)

    print(f"\n상세 정보 수집 완료: {len(results):,}개")

    # 저장
    df = pd.DataFrame(results)
    # 컬럼 이름이 'first_air_date'라고 가정
    df = df.sort_values(["release_date", "popularity"], ascending=[True, False])
    df.to_csv("files/movies_test.csv", index=False, encoding="utf-8-sig")

    elapsed = time.time() - total_start

    print("\n" + "="*90)
    print("========================== DONE ==========================")
    print(f"총 데이터: {len(df):,}개")
    print(f"소요시간: {elapsed/60:.2f}분 ({elapsed:.2f}초)")
    print(f"저장 파일: movies.csv")
    print("="*90)

NameError: name 'time' is not defined

In [5]:
if __name__ == "__main__":
    total_start = time.time()

    # 기간을 N개월 단위로 분할
    periods = tdc.generate_date_periods(start_date, end_date, months=12)

    print(f"\n전체 기간: {start_date} ~ {end_date}")
    print(f"분할 기간: {len(periods)}개")
    print("=" * 90)

    # ID 데이터를 집합으로 변환하여 중복 제거
    all_ids = set()

    for idx, (period_start, period_end) in enumerate(periods, 1):
        print(f"\n[{idx}/{len(periods)}] 기간: {period_start} ~ {period_end}")

        # 기간내 TV 시리즈 ID 수집
        current_ids_set = tdc.collect_tv_ids(period_start, period_end)
        all_ids.update(current_ids_set)

        print(f"해당 기간 내 수집된 ID: {len(current_ids_set):,}개 (누적 ID: {len(all_ids):,}개)")

    # 집합을 리스트로 변환
    id_list = sorted(list(all_ids))

    print(f"\n{'=' * 90}")
    print(f"최종 ID 개수: {len(id_list):,}개")

    # ============================================================
    # 2단계: TV 시리즈 + 시즌 상세 정보 수집 (멀티스레드)
    # ============================================================
    print(f"\n{'=' * 90}")
    print("상세 정보 멀티스레드 수집 시작...\n")

    series_results = []
    seasons_results = []

    with ThreadPoolExecutor(max_workers=20) as executor:
        # 시리즈 + 시즌 동시 수집
        futures = {
            executor.submit(tdc.fetch_tv_series_and_seasons, tv_id): tv_id
            for tv_id in id_list
        }

        for future in tqdm(as_completed(futures), total=len(futures), desc="TV 시리즈 + 시즌 수집"):
            try:
                series_data, seasons_list = future.result()

                if series_data:
                    series_results.append(series_data)

                if seasons_list:
                    seasons_results.extend(seasons_list)

            except Exception as e:
                tv_id = futures[future]
                print(f"오류 (TV ID: {tv_id}): {e}")

    print(f"\n시리즈 정보 수집 완료: {len(series_results):,}개")
    print(f"시즌 정보 수집 완료: {len(seasons_results):,}개")

    # ============================================================
    # 3단계: 저장
    # ============================================================

    # 시리즈 데이터 저장
    df_series = pd.DataFrame(series_results)
    df_series = df_series.sort_values(["first_air_date", "popularity"], ascending=[True, False])

    # 시즌 데이터 저장
    df_seasons = pd.DataFrame(seasons_results)
    df_seasons = df_seasons.sort_values(["series_id", "season_number"], ascending=[True, True])

    # CSV 저장
    df_series.to_csv("files/tv_series_2005_2015.csv", index=False, encoding="utf-8-sig")
    df_seasons.to_csv("files/tv_seasons_2005_2015.csv", index=False, encoding="utf-8-sig")

    elapsed = time.time() - total_start

    print("\n" + "=" * 90)
    print("========================== DONE ==========================")
    print(f"시리즈 데이터: {len(df_series):,}개")
    print(f"시즌 데이터: {len(df_seasons):,}개")
    print(f"소요시간: {elapsed / 60:.2f}분 ({elapsed:.2f}초)")
    print(f"저장 파일: tv_series_2005_2015.csv, tv_seasons_2005_2015.csv")
    print("=" * 90)


전체 기간: 2025-11-01 ~ 2025-11-03
분할 기간: 1개

[1/1] 기간: 2025-11-01 ~ 2025-11-03

📺 TV 시리즈 ID 수집: 2025-11-01 ~ 2025-11-03

[기간 1/1] 2025-11-01 ~ 2025-11-03
  [TV 시리즈] 총 139개 (7페이지) → 수집: 7페이지


  누적 ID: 139개

✅ 최종 수집: 139개 TV 시리즈 ID

해당 기간 내 수집된 ID: 139개 (누적 ID: 139개)

최종 ID 개수: 139개

상세 정보 멀티스레드 수집 시작...



TV 시리즈 + 시즌 수집: 100%|██████████| 139/139 [00:03<00:00, 46.30it/s]


시리즈 정보 수집 완료: 139개
시즌 정보 수집 완료: 141개

========================== DONE ==========================
시리즈 데이터: 139개
시즌 데이터: 141개
소요시간: 0.07분 (4.30초)
저장 파일: tv_series_2005_2015.csv, tv_seasons_2005_2015.csv
